In [ ]:
import numpy as np
import rp
import torch
import torch.nn as nn
import source.stable_diffusion as sd
from easydict import EasyDict
from source.learnable_textures import LearnableImageFourier
from source.stable_diffusion_labels import NegativeLabel
from itertools import chain
import time

In [ ]:
#ONLY GOOD PROMPTS HERE
example_prompts = rp.load_yaml_file('source/example_prompts.yaml')
print('Available example prompts:', ', '.join(example_prompts))

#These prompts are all strings - you can replace them with whatever you want! By default it lets you choose from example prompts
prompt_a, prompt_b, prompt_c, prompt_d, prompt_z = rp.gather(example_prompts, 'miku froggo lipstick kitten_in_box darth_vader'.split())
#Prompts a,b,c,d are the normal looking images
#Prompt z is the hidden image you get when you overlay them all on top of each other

negative_prompt = ''

print()
print('Negative prompt:',repr(negative_prompt))
print()
print('Chosen prompts:')
print('    prompt_a =', repr(prompt_a))
print('    prompt_b =', repr(prompt_b))
print('    prompt_c =', repr(prompt_c))
print('    prompt_d =', repr(prompt_d))
print('    prompt_z =', repr(prompt_z))

# New Section

In [ ]:
if 's' not in dir():
    model_name='CompVis/stable-diffusion-v1-4'      # 512 Base model 1.4
    # model_name='runwayml/stable-diffusion-v1-5'     # 512 Base model 1.5
    model_name='stabilityai/stable-diffusion-2'     # 768 Base model 2.0
    # model_name='stabilityai/stable-diffusion-2-1'   # 768 Base model 2.1
    # model_name='sd-dreambooth-library/fashion'      # 512 Fashion stuff?
    # model_name='nitrosocke/mo-di-diffusion'         # 512 Dreambooth: Use 'modern disney style' in prompt. Pixar style.
    # model_name='nitrosocke/archer-diffusion'        # 512 Dreambooth: Use 'archer style' in prompt. Thick outline clean cartoon style.
    model_name='nitrosocke/nitro-diffusion'         # 512 Dreambooth: Use 'arcane style', 'archer style' and/or 'modern disney style' in your prompt
    # model_name='Envvi/Inkpunk-Diffusion'            # 512 Dreambooth: Use 'nvinkpunk' in prompt. Looks kinda like borderlands, but more dramatic and artistic.
    # model_name='nitrosocke/classic-anim-diffusion'  # 512 Dreambooth: Use 'classic disney style' in prompt. Old timey disney style.
    # model_name='nitrosocke/Ghibli-Diffusion'        # 512 Dreambooth: Use 'ghibli style' in prompt. Studio Ghibli anime style.
    # model_name='hakurei/waifu-diffusion'            # 512 Fine Tuned: No special prompting needed
    # model_name='hakurei/artstation-diffusion'       # 512 Fine Tuned: No special prompting needed
    # model_name='prompthero/openjourney-v4'          # 512 Fine Tuned: No special prompting needed
    
    gpu=str(rp.select_torch_device())
    s=sd.StableDiffusion(gpu,model_name)
device=s.device;

In [ ]:
label_a = NegativeLabel(prompt_a + ' nvinkpunk',negative_prompt)
label_b = NegativeLabel(prompt_b + ' nvinkpunk',negative_prompt)
label_c = NegativeLabel(prompt_c + ' nvinkpunk',negative_prompt)
label_d = NegativeLabel(prompt_d + ' nvinkpunk',negative_prompt)
label_z = NegativeLabel(prompt_z + ' nvinkpunk',negative_prompt)

In [ ]:
# s.height=s.width=256
# s.height=s.width=512
s.height=s.width=768
# s.height=s.width=1024
# s.height=s.width=2048

prompt='modern disney style Closeup face portrait of a black girl wearing crown of flowers, smooth soft skin, big dreamy eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by makoto shinkai, stanley artgerm lau, wlop, rossdraws, concept art, digital painting, looking into camera'
prompt='archer style Closeup face portrait of a black girl wearing crown of flowers, smooth soft skin, big dreamy eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by makoto shinkai, stanley artgerm lau, wlop, rossdraws, concept art, digital painting, looking into camera'
prompt='arcane style, a magical princess with golden hair'
# prompt='ghibli style anime girl'
label=NegativeLabel(prompt,'ugly unrealistic blurry')
label=NegativeLabel(prompt)
image=label.get_sample_image(guidance_scale=16,num_steps=100)
rp.rinsp(image)
rp.display_image(image)

In [ ]:
s.pipe.unet.config